In [135]:
from setup import *
import pandas as pd

adj_df = pd.read_csv('Matrix.csv')
post_df = pd.read_csv('Posters.csv')

subfield_list = list(adj_df['Subfields'])
print(subfield_list)

post_response = list(post_df['RESPONSE ID'])
post_sub = list(post_df['SUBFIELD'])
post_list = [(post_response[i], post_sub[i]) for i in range(len(post_response))]
num_post = len(post_list)
print(num_post)
print(post_list[num_post-1])
print(len(post_list), len(set(post_list)))

for x in post_list:
    if x[1] not in subfield_list:
#         print('bad')
        post_list.remove(x)        

for x in post_list:
    if x[1] not in subfield_list:
#         print('bad')
        post_list.remove(x)
        
for x in post_list:
    if x[1] == 'Doctoral':
        print(x)

poster_numbered_subs = []
for x in post_list:
    poster_numbered_subs.append(subfield_list.index(x[1]))
print(poster_numbered_subs)

print(bin_edges,histo)
print(len(histo), len(bin_edges))
histo, bin_edges = np.histogram(poster_numbered_subs, bins = [x for x in range(len(subfield_list))])
hist = [(subfield_list[x],histo[x]) for x in bin_edges[:-1]]
print(hist)
sum = 0
for x in hist:
    sum+= x[1]
print(sum)

['Agriculture', 'Animal science, poultry science', 'Biology', 'Biomedical Sciences', 'Chemistry', 'Communication, Media', 'Computer Science, Engineering, Systems', 'Counseling', 'Criminal justice, Criminology, Forensics', 'Education ', 'Education - Special Ed', 'Engineering - Civil', 'Engineering - Electrical', 'Engineering - Environmental, Biological,Agricultural', 'Engineering - Mechanical', 'Engineering - Other', 'English, Literature', 'Entomology', 'Environmental Studies - Plant sciences, forestry', 'Environmental Studies - Soil sciences', 'Geography, GIS', 'Geosciences, Geology, Hydrology, Geophysics', 'Health - public, human', 'History', 'Human Resource Management, Development', 'Kinesiology, Sport Sciences', 'Language, linguistics', 'Management', 'Management Information Systems', 'Marketing', 'Math & statistics', 'Nursing', 'Organizational behavior', 'Physics', 'Political science', 'Psychology', 'Social work', 'Sociology', 'Wildlife & fisheries, management and sciences']
278
('R

In [143]:
from setup import *
from scipy.special import binom
import pandas as pd

#This is the algorithm for a SINGLE major category.  We will repeat for each category independently.

adj_df = pd.read_csv('Matrix.csv')
post_df = pd.read_csv('Posters.csv')


num_trials = 50 # number of judge->poster assignments analyzed for EACH poster->session assignment
cover = 3 # number of judgings per poster
num_post = 300 # number of posters in this category
num_judg = 90 # number of judges in this category
num_field = 40 # total numder of subfields in ALL categories.  We do not need to split subfields into categories - algorithm handles)
num_ses = 4 #number of sessions
num_univ = 8 #number of universities participating - must enumerate

#########################################################
# Code here picks random values to stand in for real data we will have after registration completes

#Randomly assign judge and poster fields
judg_field = np.random.randint(num_field, size=num_judg)
judg_univ = np.random.randint(num_univ, size=num_judg)
post_field = np.random.randint(num_field, size=num_post)
post_univ = np.random.randint(num_univ, size=num_post)

#Make random field adjency matrix
############################################################This to make sure init setup is good
bad_setup_flag = True
count = 0
print("start")
while(bad_setup_flag is True):
    print(count)
    R = np.random.rand(num_field, num_field)
    R = np.triu(R, k=1)
    R = (R + R.T)  #must be symmetric
    # print(R)
    Adj = np.zeros_like(R).astype(int)
    Adj[R>.7] = 1
    s = Adj.sum(axis=1)
#     print(s)
    P = Adj/(s[:,np.newaxis])
    if (np.isnan(P).any() == False):
#         print('Found no NaNs')
        bad_setup_flag = False
#     else:
#         print("Found a NaN")
    if (np.count_nonzero(P) <= num_field*num_field*0.25):
#         print('Not enough Adjacency')
        bad_setup_flag = True
    count += 1
        
#########################################################
# Real code starts here

### Compute field distance matrix
def show():
    print()
    print("k=%d"%k)
    print("P")
    print(P)
    print("Dist")
    print(Dist)
    

Dist = Adj.copy().astype(int)
k = 1
print("Adj")
print(Adj)
show()
count = 0
p=P.copy()
for k in range(2,5):
    # Multiply S by itself
    print(1)
    P = P.dot(p)
    print(2)
    # Write k in all entries of Dist that are 0 and the corresponding entry of A just became non-zero
    Dist[((Dist==0) & (P>0))] = k
    print(3)
    if(Dist.min() > 0): #break once Dist is full
        break
    print(4)
show()
Dist[Dist==0] = 999
Dist[np.diag_indices_from(Dist)] = 0
# print(Dist)

# Make field_incompatibility matrix num_judg x num_post, filled with Dist between judge field and poster field
field_incompat = np.fromfunction(lambda j,p: Dist[judg_field[j], post_field[p]], (num_judg,num_post), dtype=int)

# Prevent judge and poster from same univ.  Entry is True is from same univ


##################################################### This to ensure that a poster doesn't get all bad univ matches
#This will reassign university lables to the judges
for k in range(20):
    univ_match = np.fromfunction(lambda j,p: judg_univ[j] == post_univ[p], (num_judg,num_post), dtype=int)
    field_incompat[univ_match] = 999
    if (field_incompat.sum(axis = 0).any() >= 999*(int(num_judg*0.5)+0.01)): #ONLY up to half of the judges being from a pooters univ
        continue
#         print('bad university match, trying new judge-university match')
    else: 
        break

# Write 999 in corresponding entry of field_incompat
print(field_incompat)


#########################################################
# Compute judge workload per round
q = int(np.floor(num_post/num_ses)) #min number of posters per session
num_post_ses = q*np.ones(num_ses).astype(int) #make vector of length num_ses filled with q
r = num_post - q*num_ses #remainder, if num_ses does not divide num_post evenly
num_post_ses[:r] += 1 #spread these remainders across sessions
num_post_ses = num_post_ses[::-1] #reverse so that first session has fewest (for latecommers)
print(num_post_ses)
workload = np.ceil(num_post_ses.max()*cover/num_judg)
print("Judge workload = %d"%workload)


def get_post_to_ses():
    """
    Generate all possible ways to assign posters to sessions.  Two steps:
    1. Select num_ses in increasing order posters as the first posters assigned to each ses.
    2. Fill the remaining slots
    
    Step 1 prevents redundancy from earlier, more naive versions of this algorithm.
    Suppose num_ses = 2 and num_post = 4.  We do not want to see both post_ses = [[0,1],[2,3]]
    and post_ses = [[2,3],[0,1]].  These are redundant - if you flip ses 0 and ses 1,
    it is the same (except posters 2&3 present in the earlier session).    
    """
    import operator
    N = num_post
    k = num_ses
    all_posters = []
    post_list = list(range(N)) #generates all ways to select num_ses posters as first for each ses     
    for ses in range(num_ses):
        temp_list = list(np.random.choice(post_list, int(N/k), replace = False))#generates all ways to fill the rest of the slots
        post_list = list(set(post_list) - set(temp_list))        
        all_posters.append(temp_list)
        N -= int(N/k)
        k -= 1
    print()
    print()
    print(all_posters)
    return all_posters


#########################################################
#Assign judges to posters
best_post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
best_judg_post = [[]]*num_ses
best_fi_score_tot = 999999

for x in range(50):
    post_ses = get_post_to_ses()
    print(len(post_ses))
    #########################################################
    # Assign judges to posters
    best_fi_ses = [[]] * num_ses #best fi matrix
    best_fi_score_ses = [999] * num_ses #best fi_score
#     print()
#     print("Optimizing poster->session %s"%post_ses)

    for (ses, post) in enumerate(post_ses):  
        print("Optimizing session %d with posters %s"%(ses,post))
        workload = int(np.ceil(len(post)*cover/num_judg))
#         print('workload')
#         print(workload)
        for trial in range(num_trials):
            skip_trial = False
            field_incompat_sub = field_incompat[:,post] #Extract cols for posters assigned to this ses
#            print('origin')
#            print(field_incompat_sub.shape)
            temp_field_incompat_sub = field_incompat_sub
            for x in range(cover-1):
                field_incompat_sub = np.concatenate((field_incompat_sub, temp_field_incompat_sub),axis=1)
            temp_field_incompat_sub = field_incompat_sub
            for x in range(workload-1):
                field_incompat_sub = np.concatenate((field_incompat_sub, temp_field_incompat_sub),axis=0)
#             for x in range(int(workload)):
#                 field_incompat_sub = np.stack((field_incompat_sub, field_incompat_sub),axis=0)
#            print('shape')
#            print(field_incompat_sub.shape)
            fi = np.full_like(field_incompat_sub,-1)
            sum_incompat = field_incompat_sub.sum(axis = 0)
#             print('sum_incompat')
#             print(sum_incompat)
            judg_work = [0]*num_judg
            for i in range(len(post)*cover):
                if(skip_trial == True):
                    break
                col = np.argmax(sum_incompat)
                c = field_incompat_sub[:,col] #Grab column for this poster
                m = c.min()
                if(m == 999): #If no legal judge remains, abandon this trial
                    skip_trial = True
                    break
                rows = np.where(c == m)[0] #find all rows in this col where the smallest fi appears
#                 print('rows')
#                 print(rows)
                if(len(rows) == 1):
                    row = rows[0]
                else:
                    row = np.random.choice(rows) #choose one such row at random
#                       print(row)
                fi[row,col] = field_incompat_sub[row,col] #record fi for this pairing
                judg_work[row%num_judg] += 1 #increment that judge's workload tracker
                if(judg_work[row%num_judg] >= workload):
                    for i in range(workload):
                        field_incompat_sub[(row%num_judg)+i*num_judg,:] = 999 #if judge has reached workload limit, fill her row with 999
                else:
                    field_incompat_sub[row,col] = 999 #else, put 999 in the (row,col) entry only
#                         print(fi)
#                         print(field_incompat_sub)
                sum_incompat -= field_incompat_sub[row,:]
                sum_incompat[col]=(-1)
            if(skip_trial == True):
                continue
            fi_score = fi.sum() #total fi score for this judge->poster pairing for this ses
#                 print(field_incompat_sub)
#                 print(fi>0)
#                 print(fi)
#                 print(fi_score)
            if(fi_score < best_fi_score_ses[ses]): # if current beats prior best, keep it
#                 print("current ses %d fi = %d is better than prior best ses %d fi = %d.  I'll keep it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
                best_fi_ses[ses] = fi.copy()
                best_fi_score_ses[ses] = fi_score
#             else:
#                 print("current ses %d fi = %d is not better than prior best ses %d fi = %d.  I'll ignore it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
#                 print(best_fi_score_ses[ses])
            if(best_fi_score_ses[ses] <= 1):
                continue
#                 print("Found best possible j->p for ses %d.  Moving on."%ses)        
#         print("Finished optimizing session %d"%ses)
        #########################################################
        # We now have the best way to assign judges0>posters under the current poster->session assignment
        # We check if it beats the best from prior poster->session assignment
#         print("Finished optimizing poster->session %s"%post_ses)
        fi_score_tot = sum(best_fi_score_ses)
        if(fi_score_tot < best_fi_score_tot):
#             print("current total fi = %d is better than prior best total fi = %d.  I'll keep it."%(fi_score_tot,best_fi_score_tot))
            best_post_ses = post_ses.copy()
            best_judg_post = best_fi_ses.copy()
            best_fi_score_tot = fi_score_tot            
            for (ses,(ps,jp)) in enumerate(zip(best_post_ses,best_judg_post)):
#                 print(ses, (ps,jp))
                df = pd.DataFrame(np.asarray(jp))#>=0)#, columns=ps)
#                 print("session %d"%ses)
                display(df)
                print()
#         else:
#             print("current total fi = %d is not better than prior best total fi = %d.  I'll ignore it."%(fi_score_tot,best_fi_score_tot))
print(best_fi_ses)

start
0
Adj
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 1 1 0]]

k=1
P
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.11111111]
 [ 0.          0.          0.         ...,  0.          0.          0.07692308]
 [ 0.          0.          0.         ...,  0.1         0.1         0.        ]]
Dist
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 1 1 0]]
1
2
3
4
1
2
3

k=3
P
[[ 0.02250153  0.01669379  0.02468611 ...,  0.02056332  0.02813403
   0.01961174]
 [ 0.02170193  0.01821891  0.02446517 ...,  0.01787276  0.02700449
   0.01895916]
 [ 0.02917449  0.02224106  0.01931935 ...,  0.0

TypeError: 'numpy.int64' object is not callable

In [159]:
judge_counts = []
print('Workload: ' + str(workload))
print('Pairings needed: ' + str(cover*num_post/num_ses))
print(best_fi_ses)
for x in best_fi_ses:
    print('Judgings needed: ' + str(len(x)))
    small_counts = []
    single_counts = []
    for y in x:
        print(y)
        if((y==-1).sum() < len(y)):
            print((y==-1).sum(), len(y))
            small_counts.append(sum(y>=0))
        else:
            small_counts.append(-1)
    for y in range(num_judg):
        this_judge = 0
        for z in range(cover):
            this_judge += sum(x[z*y]>=0)
        single_counts.append(this_judge)
    print('Posters in session: '+ str(len(single_counts)))
    print('Judge totals: ' + single_counts)
    print('These should be 1s: ' + small_counts)
    print('Judgings counted: ' + sum(small_counts))
    judge_counts.append(small_counts)

Workload: 3
Pairings needed: 225.0
[array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ..., 
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]]), [], [], []]
Judgings needed: 270
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1

TypeError: 'numpy.int64' object is not callable